In [1]:
import torch
import gym
from gym import envs
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [ ]:
for env in envs.registry.all():
    print(env)

In [ ]:
env.observation_space

In [ ]:
env = gym.make('PongNoFrameskip-v0')
observation = env.reset()


env.render()
print(observation)
action = env.action_space.sample()
a, b, c, d = env.step(action)
print(a,b,c,d)

env.close()


In [ ]:
#Run to see visual

for i_episode in range(2):
    observation = env.reset()
    for t in range(1000):
        env.render()
        print(observation)
        action = env.action_space.sample()
        observation, reward, done, log = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


In [ ]:
env = gym.make('PongNoFrameskip-v0')
observation = env.reset()


# env.render()
print(observation)
# action = env.action_space.sample()
# a, b, c, d = env.step(action)
# print(a,b,c,d)

env.close()


In [ ]:
env.action_space

In [ ]:
type(observation)

In [ ]:
pobservation = torch.from_numpy(observation)
pobservation

In [ ]:
flattened_pobservation = pobservation.view(1,-1).float()

In [ ]:
lenobs = len(flattened_pobservation[0])
lenobs

In [ ]:
flattened_pobservation.shape

In [ ]:
preds = ANNmodel(flattened_pobservation)
preds

In [ ]:
#Sampling from actions
action = np.random.choice(np.array([0,1]), p = preds.view(2,).data.numpy())
action

In [ ]:
# Alternate model
ANNmodel2 = nn.Sequential(
           nn.Linear(lenobs, 2000),
           nn.LeakyReLU(),
           nn.Linear(2000,2),
           nn.LeakyReLU(),
           nn.Softmax(dim = 1))

In [ ]:
lenobs = 100800
ANNmodel1 = nn.Sequential(
           nn.Linear(lenobs, 2000),
           nn.Sigmoid(),
           nn.Linear(2000,2),
           nn.Sigmoid(),
           nn.Softmax(dim = 1))

In [2]:
lenobs = 100800
class ActorCritic(nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.l1 = nn.Linear(lenobs,25)
        self.l2 = nn.Linear(25,50)
        self.actor_lin1 = nn.Linear(50,2)
        self.l3 = nn.Linear(50,25)
        self.critic_lin1 = nn.Linear(25,1)

    def forward(self,x):
        x = F.normalize(x,dim=0)
        y = F.relu(self.l1(x))
        y = F.relu(self.l2(y))
        actor = F.softmax(self.actor_lin1(y),dim=0)
#         actor = F.log_softmax(self.actor_lin1(y),dim=0)
        c = F.relu(self.l3(y.detach()))
        critic = torch.tanh(self.critic_lin1(c))
        return actor, critic

In [26]:
env = gym.make('PongNoFrameskip-v0')
moveMapping = {
    0:2,
    1:3
}

model = ActorCritic()
optimizer = optim.Adam(lr=1e-4,params=ActorCritic.parameters())


reward = 0.0
for i_episode in range(1):
    values = []
    rewards = []
    logprobs = []
    observation = env.reset()
    print('---------------')
    done = False
    t = 0
    while reward == 0.0:
        t+=1
        print(t)
        pobservation = torch.from_numpy(observation)
        flattened_pobservation = pobservation.view(-1).float()
        policy, value = model(flattened_pobservation)
#         print(policy)
        values.append(value)
        action = np.random.choice(np.array([0,1]), p = policy.view(2,).data.numpy())
        logprobs.append(policy.view(-1)[action])
#         print('Action: {}'.format('right' if action==2 else 'left'))
        observation, reward, done, log = env.step(moveMapping[action])
        rewards.append(reward)
#         print('{}.Reward: {}'.format(t,reward))
#         print('---')
#         if done:
        if reward != 0.0:
            print('Episode:{} State:{} Reward:{}'.format(i_episode,t,reward))
            print("Episode finished after {} timesteps".format(t+1))    
            break
      
    # Reversing because earlier actions have greater value
    torch_values = torch.stack(values).view(-1).flip(0)
    torch_rewards = torch.Tensor(rewards).flip(0)
    torch_logprobs = torch.stack(logprobs).flip(0)
    
    
    returns = []
    gamma = 0.95
    ret = torch.Tensor([0])
    for r in torch_rewards:
        ret = r + gamma*ret
        returns.append(ret)
    returns = torch.Tensor(returns)
    returns = F.normalize(returns,dim=0)
    actor_loss = -1*torch_logprobs * (returns - torch_values.detach())
    critic_loss = torch.pow(torch_values - returns,2)
    loss = actor_loss.sum() + clc*critic_loss.sum()
    print('Starting Backpropagation')
    loss.backward()
    optimizer.step()
    print('Completed Backpropagation')
    
    

TypeError: parameters() missing 1 required positional argument: 'self'

In [ ]:
env.close()

In [ ]:
#################### Checking operations of pong #######################3

In [ ]:
 env.unwrapped.get_action_meanings()

In [ ]:
##########################################################################

In [ ]:
############################# Experimenting with Backpropagation ######################

inp = torch.tensor([2.,3.]).float()
weights = torch.tensor([1.,9.], requires_grad = True).float()
print("Params: ",weights)
gt = torch.tensor([6.])
print("Ground truth: ",gt)
out = torch.dot(inp,weights)
print("Prediction: ",out)

error = (gt - out)



In [ ]:
optimizer = optim.SGD([weights],lr = 0.1)
optimizer.zero_grad()
error.backward()
optimizer.step()

# print(ANNmodel.weight.grad)
weights.grad, out.grad

In [ ]:
#Weights after one round of backpropagation
weights

In [ ]:
# Checking variables with requires_grad
inp.requires_grad, weights.requires_grad, gt.requires_grad, out.requires_grad, error.requires_grad

In [ ]:
###############################################################################################

In [ ]:
############################ Manually setting gradients #######################################

In [ ]:
MinorModel2 = nn.Sequential( 
             nn.Linear(2,1, bias = False))

inp = torch.tensor([2.,3.])
out = MinorModel(inp)
out

In [ ]:
for params in MinorModel2.parameters():
    print(params)

In [ ]:
MinorModel2[0].weight

In [ ]:
MinorModel2[0].weight.grad = torch.Tensor([1,2]).view(1,-1)

In [ ]:
o2 = optim.SGD(MinorModel2.parameters(), lr=0.1)

o2.step()

In [ ]:
for params in MinorModel2.parameters():
    print(params)

In [ ]:
########################################################

In [ ]:
####################### Testing ########################

In [ ]:

MinorModel = nn.Sequential( 
             nn.Linear(2,1, bias = False))

inp = torch.tensor([2.,3.])
out = MinorModel(inp)
out


In [ ]:

for name, params in MinorModel.named_parameters():
    print(name, params)
    


In [ ]:

for params in MinorModel.parameters():
    print(params)
    print(type(params))
    
print(MinorModel.parameters())


In [ ]:
#######################################################